In [ ]:
# CSV的阅读和处理
# 导入各种各样的包：
import pandas as pd
import numpy as np
import json
import os
import re
# 这是一个数据读取的脚本，如果你需要使用，请务必自己调整文件地址！！！！

In [3]:
with open('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2.csv', 'rb') as f:
    import chardet
    result = chardet.detect(f.read(10000))
    encoding = result['encoding']
    print(f"文件编码为：{encoding}")

文件编码为：GB2312


In [5]:
df = pd.read_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2.csv', encoding='gb18030', low_memory=False)
#打印columns:
print(df.columns)

Index(['comment_id', 'username', 'user_id', 'score', 'content', 'time', 'city',
       'gender', 'approve_count', 'reply_count', 'user_level', 'vip_type',
       'is_vip', 'avatar_url', 'tags', 'is_purchase', 'is_hot'],
      dtype='object')


In [8]:
#重构CSV，我们将会丢去一些列：'comment_id', 'username', 'user_id', 'approve_count', 'reply_count', 'user_level', 'vip_type', 'is_vip', 'avatar_url', 'tags', 'is_purchase', 'is_hot'

# 要丢弃的列
columns_to_drop = [
    'username', 'user_id', 'approve_count', 'reply_count',
    'user_level', 'vip_type', 'is_vip', 'avatar_url', 'tags',
    'is_purchase', 'is_hot'
]

# 检查这些列是否都存在于当前 DataFrame 中（防止报错）
columns_to_drop = [col for col in columns_to_drop if col in df.columns]

# 丢掉不需要的列
df_restructured = df.drop(columns=columns_to_drop)

# 查看前几行，确认结构
print("\n重构后的数据：")
print(df_restructured.head())

# 可选择保存为新文件（以防原文件被覆盖）
df_restructured.to_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_dropped_unnecessary_cols.csv', index=False, encoding='utf-8-sig')
print("\n✅ 重构后的 CSV 已保存为 'Nezha2_dropped_unnecessary_cols.csv'")



重构后的数据：
   comment_id  score   content             time city gender
0  1211665559    5.0   二刷！依旧精彩  2025/4/21 21:16   佛山     未知
1  1211652698    5.0  支持一下国产动漫  2025/4/21 21:15   延边      1
2  1211623835    5.0   剧情搞笑又紧张  2025/4/21 21:13   武汉     未知
3  1211642772    3.5  还行吧，特效不错  2025/4/21 21:08   深圳     未知
4  1211659669    4.5    值得看的一部  2025/4/21 20:59   海东     未知

✅ 重构后的 CSV 已保存为 'Nezha2_dropped_unnecessary_cols.csv'


In [4]:
# 接下来，我们在新数据集上操作：
df = pd.read_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_dropped_unnecessary_cols.csv', encoding='utf-8-sig', low_memory=False)
#打印columns:
print(df.columns)

Index(['comment_id', 'score', 'content', 'time', 'city', 'gender'], dtype='object')


In [6]:
#情感分析：
from snownlp import SnowNLP
from tqdm import tqdm
tqdm.pandas(desc="正在分析情感")
# 检查是否有空值，并填充为 None
df['content'] = df['content'].replace('', np.nan)
# 使用 SnowNLP 进行情感分析
df['sentiment'] = df['content'].progress_apply(lambda x: SnowNLP(str(x)).sentiments if pd.notnull(x) else None)

# 显示前几行检查结果
print(df[['content', 'sentiment']].head())

# 保存带情感分析结果的新 CSV
output_path = 'C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_now_with_sentiment.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 情感分析完成并保存为：{output_path}")

正在分析情感: 100%|██████████| 1326/1326 [00:11<00:00, 115.00it/s]

    content  sentiment
0   二刷！依旧精彩   0.797696
1  支持一下国产动漫   0.482468
2   剧情搞笑又紧张   0.633187
3  还行吧，特效不错   0.947526
4    值得看的一部   0.852965

✅ 情感分析完成并保存为：C:\Users\顾子琪\Desktop\上纽大文件\2025 SPRING\Nezha2_now_with_sentiment.csv


In [7]:
#好吧，这个东西看起来不是特别的准，我们换一个工具包：
#GPT： 使用HuggingFace Transformers + 中文 BERT 情感模型
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 载入模型（示例模型名，可换）
model_name = "IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

text = "二刷！依旧精彩"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
probs = torch.softmax(outputs.logits, dim=1)
print("情感倾向:", probs)


vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

c:\Users\顾子琪\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\顾子琪\.cache\huggingface\hub\models--IDEA-CCNL--Erlangshen-Roberta-110M-Sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


情感倾向: tensor([[0.0087, 0.9913]], grad_fn=<SoftmaxBackward0>)


model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

In [8]:
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 读取数据
df = pd.read_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_dropped_unnecessary_cols.csv', encoding='utf-8-sig', low_memory=False)

# 加载模型和 tokenizer
tokenizer = AutoTokenizer.from_pretrained("IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment")
model = AutoModelForSequenceClassification.from_pretrained("IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment")

# 放到GPU如果有的话
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 情感分析函数
def get_positive_score(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.0
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        positive_score = probs[0][1].item()  # 正面情绪概率
    return round(positive_score, 4)

# 用 tqdm 处理所有评论
tqdm.pandas(desc="Analyzing Sentiment")
df["sentiment"] = df["content"].progress_apply(get_positive_score)

# 显示前几行检查结果
print(df[['content', 'sentiment']].head())

# 保存结果
output_path = 'C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_sentiment_by_advancedNLPs.csv'
df.to_csv(output_path, encoding='utf-8-sig', index=False)

print("分析完成，文件已保存为：", output_path)



Analyzing Sentiment: 100%|██████████| 1326/1326 [01:48<00:00, 12.21it/s]

    content  sentiment
0   二刷！依旧精彩     0.9913
1  支持一下国产动漫     0.9101
2   剧情搞笑又紧张     0.9853
3  还行吧，特效不错     0.9776
4    值得看的一部     0.9496
分析完成，文件已保存为： C:\Users\顾子琪\Desktop\上纽大文件\2025 SPRING\Nezha2_sentiment_by_advancedNLPs.csv


In [ ]:
# 哦对，换了先进的NLP工具包之后，准确度上升了不少。
"""
snow NLP说：
1211618654	1.5	剧情逻辑、人物塑造不能细想	2025/4/21 10:44	北京	1	0.7763131252505957

但是advanced NLP这样说：
1211618654	1.5	剧情逻辑、人物塑造不能细想	2025/4/21 10:44	北京	1	0.0027
"""

#这就很对了！

In [14]:
#OK,接下来，把我们的城市转变为省份：

#导包：
import pandas as pd
import cpca
from tqdm import tqdm

# 读取数据
df = pd.read_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_sentiment_by_advancedNLPs.csv', encoding='utf-8-sig', low_memory=False)

# 初始化 tqdm 进度条
tqdm.pandas(desc="解析城市到省份")

# 使用 cpca 时，需要将城市列转换为 pandas Series 类型
df['province'] = cpca.transform(df['city'].values)['省']

# 填充“未知省份”
df['province'] = df['province'].fillna('未知省份')

# 打印合法省份及评论数量（去除“未知省份”）
province_counts = df[df['province'] != '未知省份']['province'].value_counts()

print("\n✅ 合法省份及评论数量：")
for province, count in province_counts.items():
    print(f"{province}: {count} 条评论")

# 保存带省份的 CSV
df.to_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_with_province.csv', encoding='utf-8-sig', index=False)

print("\n🎉 文件已保存为 Nezha2_with_province.csv")





✅ 合法省份及评论数量：
广东省: 154 条评论
江苏省: 144 条评论
山东省: 142 条评论
北京市: 77 条评论
浙江省: 77 条评论
广西壮族自治区: 64 条评论
山西省: 58 条评论
四川省: 52 条评论
辽宁省: 48 条评论
河南省: 45 条评论
湖北省: 39 条评论
河北省: 37 条评论
陕西省: 36 条评论
安徽省: 32 条评论
湖南省: 30 条评论
内蒙古自治区: 22 条评论
云南省: 21 条评论
福建省: 20 条评论
贵州省: 19 条评论
上海市: 18 条评论
重庆市: 17 条评论
黑龙江省: 17 条评论
新疆维吾尔自治区: 16 条评论
甘肃省: 14 条评论
吉林省: 11 条评论
天津市: 11 条评论
宁夏回族自治区: 10 条评论
江西省: 9 条评论
海南省: 9 条评论
青海省: 6 条评论
西藏自治区: 2 条评论

🎉 文件已保存为 Nezha2_with_province.csv


In [ ]:
# cpca坏了吗？
# 取出几个示例城市
sample_cities = ['北京', '上海', '广州', '深圳', '杭州']

sample_province = cpca.transform(sample_cities)
print(sample_province)



     省     市     区 地址  adcode
0  北京市  None  None     110000
1  上海市  None  None     310000
2  广东省   广州市  None     440100
3  广东省   深圳市  None     440300
4  浙江省   杭州市  None     330100


In [ ]:
#OK,所有的东西均已经完成，我们会最终输出一个完整版本的csv文件，包含了：
"""
comment_id 评论ID
score 评分
content 评论内容
time 评论时间
city 评论城市
gender 性别
sentiment 情感倾向
province 省份
"""

In [15]:
# reading CSV
df = pd.read_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_with_province.csv', encoding='utf-8-sig', low_memory=False)
# 打印columns:
print(df.columns)
df.to_csv('C:\\Users\\顾子琪\\Desktop\\上纽大文件\\2025 SPRING\\Nezha2_fin.csv', encoding='utf-8-sig', index=False)
print("\n🎉 文件已保存为 Nezha2_fin.csv")

Index(['comment_id', 'score', 'content', 'time', 'city', 'gender', 'sentiment',
       'province'],
      dtype='object')

🎉 文件已保存为 Nezha2_fin.csv
